# Download Lansat Data From Google Cloud



## This notebook is based on Bruno's tutorial (link #3) with addition to download the Landsat data from Google Storage instead of Amazon S3 

### Links

1. Create dataframe and make it a geodataframe: https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html

2. Tutorial to create bounding box around a geodataframe: http://geologyandpython.com/geology-of-MI-and-WI.html

3. Get Landsat Reference System & Intersect it with the bounding box: http://geologyandpython.com/get-landsat-8.html

In [2]:
import pandas as pd
import geopandas as gpd
import folium
import os, shutil
from glob import glob

## STEPS

### 1. Create Dataframe with Points

Tip: Get bouning box coordinates using geojson.io

In [3]:
# Create a dataframe with lat & long. Replace with yours
df = pd.DataFrame({'Point': ['1', '2', '3', '4', '5'],
     'Latitude': [-13.643651383348415, -13.643651383348415, -13.501480338014135,-13.501480338014135, -13.643651383348415],
     'Longitude': [48.290748596191406, 48.575706481933594, 48.575706481933594, 48.290748596191406, 48.290748596191406]})



### 2. Convert to Geodataframe

A GeoDataFrame needs a shapely object. We use geopandas points_from_xy() to transform Longitude and Latitude into a list of shapely.Point objects and set it as a geometry while creating the GeoDataFrame. (note that points_from_xy() is an enhanced wrapper for [Point(x, y) for x, y in zip(df.Longitude, df.Latitude)])


In [4]:
study_area_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

#The GeoDataFrame (gdf) looks like this :
print(study_area_gdf.head())

  Point   Latitude  Longitude                    geometry
0     1 -13.643651  48.290749  POINT (48.29075 -13.64365)
1     2 -13.643651  48.575706  POINT (48.57571 -13.64365)
2     3 -13.501480  48.575706  POINT (48.57571 -13.50148)
3     4 -13.501480  48.290749  POINT (48.29075 -13.50148)
4     5 -13.643651  48.290749  POINT (48.29075 -13.64365)


### 3. Export to CSV

In [4]:
# Save it to CSV file
study_area_gdf.to_csv('YOUR_FILE.csv')


### 4. Get the Bounding Box & Save it
.envelope is a shapely method to get the bounding rectangle of all the points in the geometry.
Let's save the bounds for the next parts of Integrating & Exploring.

In [5]:
from shapely import geometry

multipoints = geometry.MultiPoint(study_area_gdf['geometry'])
bounds = multipoints.envelope

# Let's save the bounds for the next parts
gpd.GeoSeries(bounds).to_file('YOURFILE.gpkg', 'GPKG')

### 5. Download WRS Landsat Ascending
The Worldwide Reference System (WRS) is a global notation system for Landsat data.
Download WRS2 ascending data

Page: https://www.usgs.gov/land-resources/nli/landsat/landsat-shapefiles-and-kml-files


In [5]:
WRS_PATH = 'PATH_TO_WRS2_descending_0.zip'
LANDSAT_PATH = os.path.dirname(WRS_PATH)

# Unzip
shutil.unpack_archive(WRS_PATH, os.path.join(LANDSAT_PATH, 'wrs2'))

In [7]:
# Import it to a GeoDataFrame using geopandas. Check the first 5 entries. Parth & Row is important for us.

wrs = gpd.GeoDataFrame.from_file('PATH_TO_WRS2_descending.shp')
wrs.head()

,AREA,PERIMETER,PR_,PR_ID,RINGS_OK,RINGS_NOK,PATH,ROW,MODE,SEQUENCE,WRSPR,PR,ACQDayL7,ACQDayL8,geometry
0,15.74326,26.98611,1.0,1.0,1,0,13,1,D,2233,013001,013001,1,9,"POLYGON ((-10.80341 80.98880, -8.97407 80.3420..."
1,14.55366,25.84254,2.0,2.0,1,0,13,2,D,2234,013002,013002,1,9,"POLYGON ((-29.24250 80.18681, -29.29593 80.198..."
2,13.37247,24.20303,3.0,3.0,1,0,13,3,D,2235,013003,013003,1,9,"POLYGON ((-24.04206 79.12261, -23.78294 79.063..."
3,12.26691,22.40265,4.0,4.0,1,0,13,4,D,2236,013004,013004,1,9,"POLYGON ((-36.66813 77.46094, -40.05219 78.098..."
4,11.26511,20.64284,5.0,5.0,1,0,13,5,D,2237,013005,013005,1,9,"POLYGON ((-44.11210 76.93656, -44.12470 76.938..."


### 6. Intersect Bounding Box with WRS-2 system

In [100]:
# Import Bounding box & Intersect

bounds = gpd.read_file('YOURFILE.gpkg')

wrs_intersection = wrs[wrs.intersects(bounds.geometry[0])]

# Also, let's put the calculated paths and rows in a variable.
paths, rows = wrs_intersection['PATH'].values, wrs_intersection['ROW'].values


### 7. Create a map to visualize the paths and rows that intersects.

Click on the Polygon to get the path and row value.


In [101]:
import numpy as np

# Get the center of the map
xy = np.asarray(bounds.centroid[0].xy).squeeze()
center = list(xy[::-1])

# Select a zoom
zoom = 9

# Create the most basic OSM folium map
m = folium.Map(location=center, zoom_start=zoom, control_scale=True)

# Add the bounds GeoDataFrame in red
m.add_child(folium.GeoJson(bounds.__geo_interface__, name='Area of Study', 
                           style_function=lambda x: {'color': 'red', 'alpha': 0}))

# Iterate through each Polygon of paths and rows intersecting the area
for i, row in wrs_intersection.iterrows():
    # Create a string for the name containing the path and row of this Polygon
    name = 'path: %03d, row: %03d' % (row.PATH, row.ROW)
    # Create the folium geometry of this Polygon 
    g = folium.GeoJson(row.geometry.__geo_interface__, name=name)
    # Add a folium Popup object with the name string
    g.add_child(folium.Popup(name))
    # Add the object to the map
    g.add_to(m)

folium.LayerControl().add_to(m)
#m.save('./images/10/wrs.html')
m

### 7.1 Print Path & Row for the scene

In [102]:
for i, (path, row) in enumerate(zip(paths, rows)):
    print('Image', i+1,',','Path is:',path,',','Row is:',row)

Image 1 , Path is: 159 , Row is: 69


### 8. Get Google Scenes From index.csv

To help locate data of interest, an index CSV file of the Landsat data is available. This CSV file lists basic properties of the available images, including their acquisition dates and their spatial extent as minimum and maximum latitudes and longitudes. The file is found in the Landsat Cloud Storage bucket:

gs://gcp-public-data-landsat/index.csv.gz

In [11]:
google_scenes = pd.read_csv('https://storage.googleapis.com/gcp-public-data-landsat/index.csv.gz', compression='gzip')


In [103]:
google_scenes.head(10)

,SCENE_ID,PRODUCT_ID,SPACECRAFT_ID,SENSOR_ID,DATE_ACQUIRED,COLLECTION_NUMBER,COLLECTION_CATEGORY,SENSING_TIME,DATA_TYPE,WRS_PATH,WRS_ROW,CLOUD_COVER,NORTH_LAT,SOUTH_LAT,WEST_LON,EAST_LON,TOTAL_SIZE,BASE_URL
0,LC81070762013196LGN01,LC08_L1TP_107076_20130715_20170503_01_T1,LANDSAT_8,OLI_TIRS,2013-07-15,01,T1,2013-07-15T01:34:09.0544310Z,L1TP,107,76,0.00,-22.05547,-24.16897,125.40929,127.66620,912213234,gs://gcp-public-data-landsat/LC08/01/107/076/L...
1,LC82320102018253LGN00,LC08_L1GT_232010_20180910_20180910_01_RT,LANDSAT_8,OLI_TIRS,2018-09-10,01,RT,2018-09-10T13:58:00.7586010Z,L1GT,232,10,99.98,72.06331,69.69304,-35.65109,-28.12479,766646647,gs://gcp-public-data-landsat/LC08/01/232/010/L...
2,LT51110851995303ASA00,LT05_L1GS_111085_19951030_20170106_01_T2,LANDSAT_5,TM,1995-10-30,01,T2,1995-10-30T01:00:01.7970940Z,L1GS,111,85,100.00,-35.07360,-37.01004,115.64894,118.31960,81322708,gs://gcp-public-data-landsat/LT05/01/111/085/L...
3,LE70910772011070ASA00,NaN,LANDSAT_7,ETM,2011-03-11,PRE,NaN,2011-03-11T23:47:14.7054141Z,L1GT,91,77,100.00,-23.58547,-25.50247,149.70145,152.10403,68238761,gs://gcp-public-data-landsat/LE07/PRE/091/077/...
4,LE70160142016231EDC01,LE07_L1TP_016014_20160818_20161007_01_T1,LANDSAT_7,ETM,2016-08-18,01,T1,2016-08-18T15:47:41.9747452Z,L1TP,16,14,8.00,66.69347,64.43492,-67.81251,-61.82216,213477257,gs://gcp-public-data-landsat/LE07/01/016/014/L...
5,LT51570511998041XXX01,LT05_L1GS_157051_19980210_20161225_01_T2,LANDSAT_5,TM,1998-02-10,01,T2,1998-02-10T06:07:15.4020880Z,L1GS,157,51,1.00,13.95482,12.05175,56.13438,58.28679,71168200,gs://gcp-public-data-landsat/LT05/01/157/051/L...
6,LT50010632003354CUB00,NaN,LANDSAT_5,TM,2003-12-20,PRE,NaN,2003-12-20T14:10:31.8000380Z,L1G,1,63,25.00,-3.39416,-5.27718,-66.59488,-64.48164,172177279,gs://gcp-public-data-landsat/LT05/PRE/001/063/...
7,LT50920872004338ASA00,NaN,LANDSAT_5,TM,2004-12-03,PRE,NaN,2004-12-03T23:49:19.0910880Z,L1T,92,87,90.00,-37.91194,-39.85877,144.02533,146.81334,98184206,gs://gcp-public-data-landsat/LT05/PRE/092/087/...
8,LT50900782010100ASA00,LT05_L1TP_090078_20100410_20161015_01_T1,LANDSAT_5,TM,2010-04-10,01,T1,2010-04-10T23:38:48.6200750Z,L1TP,90,78,39.00,-25.04632,-26.95426,150.87979,153.28210,182140288,gs://gcp-public-data-landsat/LT05/01/090/078/L...
9,LT51720421990140RSA00,LT05_L1TP_172042_19900520_20171209_01_T1,LANDSAT_5,TM,1990-05-20,01,T1,1990-05-20T07:20:34.9000690Z,L1TP,172,42,0.00,26.97900,25.02140,35.76218,38.15335,137566411,gs://gcp-public-data-landsat/LT05/01/172/042/L...


### 9. Filter Scenes

The scenes can be filtered so that you can get the desired images. Filtering SPACECRAFT_ID, CLOUD_COVER can be very useful.

If your study area yields only one scene (in my case Image 1 , Path is: 159 , Row is: 69) follow **step 9.1**
Else, if you have more than one scenes **jump to 9.2 (This step is not tested)**

### 9.1 Filter A Single Scene 

### Data Types

**Landsat 7 Data Type:** L1TP. Level 1 Terrain (Corrected) (L1TP) product — Includes radiometric, geometric, and precision correction, and uses a DEM to correct parallax errors due to local topographic relief.

**Read more about landsat collection 1:** https://www.usgs.gov/land-resources/nli/landsat/landsat-collection-1?qt-science_support_page_related_con=1#qt-science_support_page_related_con


In [108]:
# Landsat 5: T1, Cloud cover < 40
filtered_google_scene = google_scenes.loc[(google_scenes.SPACECRAFT_ID == "LANDSAT_5") & 
                                          (google_scenes.CLOUD_COVER <= 40) &
                                          (google_scenes.COLLECTION_CATEGORY == "T1") &
                                          (google_scenes.WRS_PATH == YOUR_PATH_NUMBER) &
                                          (google_scenes.WRS_ROW == YOUR_ROW_NUMBER)
                                         ]

In [109]:
filtered_google_scene.head(20)

,SCENE_ID,PRODUCT_ID,SPACECRAFT_ID,SENSOR_ID,DATE_ACQUIRED,COLLECTION_NUMBER,COLLECTION_CATEGORY,SENSING_TIME,DATA_TYPE,WRS_PATH,WRS_ROW,CLOUD_COVER,NORTH_LAT,SOUTH_LAT,WEST_LON,EAST_LON,TOTAL_SIZE,BASE_URL
6155,LT51590692000301JSA00,LT05_L1TP_159069_20001027_20161213_01_T1,LANDSAT_5,TM,2000-10-27,01,T1,2000-10-27T06:29:30.8580000Z,L1TP,159,69,19.0,-12.05592,-13.96822,47.38139,49.54557,133837548,gs://gcp-public-data-landsat/LT05/01/159/069/L...
25871,LT51590692010344MLK00,LT05_L1TP_159069_20101210_20161011_01_T1,LANDSAT_5,TM,2010-12-10,01,T1,2010-12-10T06:40:34.0180380Z,L1TP,159,69,21.0,-12.07245,-13.96311,47.40364,49.59784,140220928,gs://gcp-public-data-landsat/LT05/01/159/069/L...
37248,LT51590691987185XXX01,LT05_L1TP_159069_19870704_20170212_01_T1,LANDSAT_5,TM,1987-07-04,01,T1,1987-07-04T06:15:43.0340250Z,L1TP,159,69,7.0,-12.07822,-13.96577,47.43132,49.58954,109253103,gs://gcp-public-data-landsat/LT05/01/159/069/L...
74414,LT51590692004168JSA00,LT05_L1TP_159069_20040616_20161130_01_T1,LANDSAT_5,TM,2004-06-16,01,T1,2004-06-16T06:32:13.1110630Z,L1TP,159,69,8.0,-12.06953,-13.96299,47.38701,49.57856,112932319,gs://gcp-public-data-landsat/LT05/01/159/069/L...
739950,LT51590691994348XXX03,LT05_L1TP_159069_19941214_20170111_01_T1,LANDSAT_5,TM,1994-12-14,01,T1,1994-12-14T06:04:26.2840560Z,L1TP,159,69,28.0,-12.07291,-13.97127,47.43955,49.60335,111031951,gs://gcp-public-data-landsat/LT05/01/159/069/L...
1113168,LT51590692003325JSA00,LT05_L1TP_159069_20031121_20161204_01_T1,LANDSAT_5,TM,2003-11-21,01,T1,2003-11-21T06:29:10.1500190Z,L1TP,159,69,23.0,-12.07730,-13.96547,47.35925,49.53994,131330642,gs://gcp-public-data-landsat/LT05/01/159/069/L...
1358100,LT51590692006189JSA00,LT05_L1TP_159069_20060708_20161120_01_T1,LANDSAT_5,TM,2006-07-08,01,T1,2006-07-08T06:43:41.2470940Z,L1TP,159,69,20.0,-12.06426,-13.95754,47.39818,49.57308,122933018,gs://gcp-public-data-landsat/LT05/01/159/069/L...
1389176,LT51590692000205JSA00,LT05_L1TP_159069_20000723_20161213_01_T1,LANDSAT_5,TM,2000-07-23,01,T1,2000-07-23T06:27:51.2290500Z,L1TP,159,69,29.0,-12.02870,-13.94102,47.37072,49.53193,125848100,gs://gcp-public-data-landsat/LT05/01/159/069/L...
1396268,LT51590692008227JSA00,LT05_L1TP_159069_20080814_20161030_01_T1,LANDSAT_5,TM,2008-08-14,01,T1,2008-08-14T06:36:42.4820750Z,L1TP,159,69,17.0,-12.05909,-13.94967,47.41771,49.61996,127218935,gs://gcp-public-data-landsat/LT05/01/159/069/L...
1655075,LT51590691997116JSA00,LT05_L1TP_159069_19970426_20161231_01_T1,LANDSAT_5,TM,1997-04-26,01,T1,1997-04-26T06:18:27.4270000Z,L1TP,159,69,6.0,-12.05371,-13.96033,47.42032,49.58692,113297372,gs://gcp-public-data-landsat/LT05/01/159/069/L...


### 9.2 Filter Multiple Scenes (Not Tested)

In [ ]:
# Iterate through paths and rows (from step 6)
for path, row in zip(paths, rows):

    print('Path:',path, 'Row:', row)

    # Filter the Landsat Google Cloud table for images matching path, row, cloudcover and processing state.
    filtered_multiple_scenes = google_scenes.loc[(google_scenes.SPACECRAFT_ID == "LANDSAT_5") & 
                                              (google_scenes.CLOUD_COVER <= 40) &
                                              (google_scenes.COLLECTION_CATEGORY == "T1") &
                                              (google_scenes.WRS_PATH == path) &
                                              (google_scenes.WRS_ROW == row)
                                             ]
    
    

In [ ]:
filtered_google_scene.head(20)

### 10. Export Filtered Scenes

Make sure to use the correct dataframe name either from step 9.1 or 9.2

In [88]:
# Export Filtered Scenes to CSV
filtered_google_scene.to_csv('YOUR_FILTERED_FILE.csv')

In [89]:
# Read CSV as dataframe
L7 = pd.read_csv('YOUR_FILTERED_FILE.csv')
L7

,Unnamed: 0,SCENE_ID,PRODUCT_ID,SPACECRAFT_ID,SENSOR_ID,DATE_ACQUIRED,COLLECTION_NUMBER,COLLECTION_CATEGORY,SENSING_TIME,DATA_TYPE,WRS_PATH,WRS_ROW,CLOUD_COVER,NORTH_LAT,SOUTH_LAT,WEST_LON,EAST_LON,TOTAL_SIZE,BASE_URL
0,6155,LT51590692000301JSA00,LT05_L1TP_159069_20001027_20161213_01_T1,LANDSAT_5,TM,2000-10-27,1,T1,2000-10-27T06:29:30.8580000Z,L1TP,159,69,19.0,-12.05592,-13.96822,47.38139,49.54557,133837548,gs://gcp-public-data-landsat/LT05/01/159/069/L...
1,25871,LT51590692010344MLK00,LT05_L1TP_159069_20101210_20161011_01_T1,LANDSAT_5,TM,2010-12-10,1,T1,2010-12-10T06:40:34.0180380Z,L1TP,159,69,21.0,-12.07245,-13.96311,47.40364,49.59784,140220928,gs://gcp-public-data-landsat/LT05/01/159/069/L...
2,37248,LT51590691987185XXX01,LT05_L1TP_159069_19870704_20170212_01_T1,LANDSAT_5,TM,1987-07-04,1,T1,1987-07-04T06:15:43.0340250Z,L1TP,159,69,7.0,-12.07822,-13.96577,47.43132,49.58954,109253103,gs://gcp-public-data-landsat/LT05/01/159/069/L...
3,74414,LT51590692004168JSA00,LT05_L1TP_159069_20040616_20161130_01_T1,LANDSAT_5,TM,2004-06-16,1,T1,2004-06-16T06:32:13.1110630Z,L1TP,159,69,8.0,-12.06953,-13.96299,47.38701,49.57856,112932319,gs://gcp-public-data-landsat/LT05/01/159/069/L...
4,739950,LT51590691994348XXX03,LT05_L1TP_159069_19941214_20170111_01_T1,LANDSAT_5,TM,1994-12-14,1,T1,1994-12-14T06:04:26.2840560Z,L1TP,159,69,28.0,-12.07291,-13.97127,47.43955,49.60335,111031951,gs://gcp-public-data-landsat/LT05/01/159/069/L...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,14561007,LT51590691994220JSA00,LT05_L1TP_159069_19940808_20170113_01_T1,LANDSAT_5,TM,1994-08-08,1,T1,1994-08-08T06:08:14.3160560Z,L1TP,159,69,5.0,-12.06725,-13.96843,47.42019,49.58133,113221519,gs://gcp-public-data-landsat/LT05/01/159/069/L...
78,15146064,LT51590691996354JSA00,LT05_L1TP_159069_19961219_20170101_01_T1,LANDSAT_5,TM,1996-12-19,1,T1,1996-12-19T06:13:09.2760690Z,L1TP,159,69,32.0,-12.06190,-13.97121,47.42569,49.59238,154695791,gs://gcp-public-data-landsat/LT05/01/159/069/L...
79,15233663,LT51590691996210JSA00,LT05_L1TP_159069_19960728_20170103_01_T1,LANDSAT_5,TM,1996-07-28,1,T1,1996-07-28T06:05:49.9700810Z,L1TP,159,69,29.0,-12.06176,-13.96840,47.41465,49.57585,122457072,gs://gcp-public-data-landsat/LT05/01/159/069/L...
80,15396645,LT51590692005266JSA00,LT05_L1TP_159069_20050923_20161124_01_T1,LANDSAT_5,TM,2005-09-23,1,T1,2005-09-23T06:38:52.4450380Z,L1TP,159,69,7.0,-12.06632,-13.95734,47.34829,49.53999,120254742,gs://gcp-public-data-landsat/LT05/01/159/069/L...


# Let's look closer to the data

Check the first scene properties. In my case

**Product ID:** 
LE07_L1TP_159069_20191024_20191025_01_RT

**Product Base URL:**
gs://gcp-public-data-landsat/LE07/01/159/069/LE07_L1TP_159069_20191024_20191025_01_RT

When I open the Base_Url automatic download doesnt start (perhaps becuase its an uri and not an url)

**Find the same imagery in Google Cloud:**
https://console.cloud.google.com/storage/browser/gcp-public-data-landsat?pli=1

You don't have to do this. This step is to retrieve the actual download URL link. 
In the web page follow the links until you get to your scene. I followed to find a Landsat-7 scene.
Select: LE07/ & 01/ & (My Path) 159/& (My Row) 069/ & LE07_L1TP_159069_20191024_20191025_01_RT
Here I can see the actual links.

**Scene URL with all Bands:** 
https://console.cloud.google.com/storage/browser/gcp-public-data-landsat/LE07/01/159/069/LE07_L1TP_159069_20191024_20191025_01_RT/?pli=1

Works without the "/?pli=1" in the end

**A single band URL:** This DOWNLOADS the file
https://storage.cloud.google.com/gcp-public-data-landsat/LE07/01/159/069/LE07_L1TP_159069_20191024_20191025_01_RT/LE07_L1TP_159069_20191024_20191025_01_RT_B1.TIF

**A single band URI:** (This link doesnt automatically download data use URL)
gs://gcp-public-data-landsat/LE07/01/159/069/LE07_L1TP_159069_20191024_20191025_01_RT/LE07_L1TP_159069_20191024_20191025_01_RT_B1.TIF

### How to change the Base_Url to get the direct download links?

1. Split Base_Url & replace "gs://" with "https://storage.cloud.google.com/"
2. Add: "/" + PRODUCT_ID + "_B" 
3. Later on add (i) + ".TIF" to loop through all bands

### 11. Edit Dataframe Column

In [90]:
# Read CSV as dataframe
L7 = pd.read_csv("YOUR_FILTERED_FILE.csv")

# Split the Base_URL column
new = L7["BASE_URL"].str.split("gs://", n = 1, expand = True)

# Create a separate url column & add "/" + PRODUCT_ID + "_B" : /LE07_L1TP_159069_20191024_20191025_01_RT_B
L7["url"]= 'https://storage.googleapis.com/' + new[1] + "/" + L7["PRODUCT_ID"] + "_B"


# Dropping old BASE_URL columns 
#data.drop(columns =["BASE_URL"], inplace = True) 

# Update CSV or Create a new one
L7.to_csv('YOUR_FILTERED_FILE_Updated.csv')

  
# df display 
L7 

,Unnamed: 0,SCENE_ID,PRODUCT_ID,SPACECRAFT_ID,SENSOR_ID,DATE_ACQUIRED,COLLECTION_NUMBER,COLLECTION_CATEGORY,SENSING_TIME,DATA_TYPE,WRS_PATH,WRS_ROW,CLOUD_COVER,NORTH_LAT,SOUTH_LAT,WEST_LON,EAST_LON,TOTAL_SIZE,BASE_URL,url
0,6155,LT51590692000301JSA00,LT05_L1TP_159069_20001027_20161213_01_T1,LANDSAT_5,TM,2000-10-27,1,T1,2000-10-27T06:29:30.8580000Z,L1TP,159,69,19.0,-12.05592,-13.96822,47.38139,49.54557,133837548,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
1,25871,LT51590692010344MLK00,LT05_L1TP_159069_20101210_20161011_01_T1,LANDSAT_5,TM,2010-12-10,1,T1,2010-12-10T06:40:34.0180380Z,L1TP,159,69,21.0,-12.07245,-13.96311,47.40364,49.59784,140220928,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
2,37248,LT51590691987185XXX01,LT05_L1TP_159069_19870704_20170212_01_T1,LANDSAT_5,TM,1987-07-04,1,T1,1987-07-04T06:15:43.0340250Z,L1TP,159,69,7.0,-12.07822,-13.96577,47.43132,49.58954,109253103,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
3,74414,LT51590692004168JSA00,LT05_L1TP_159069_20040616_20161130_01_T1,LANDSAT_5,TM,2004-06-16,1,T1,2004-06-16T06:32:13.1110630Z,L1TP,159,69,8.0,-12.06953,-13.96299,47.38701,49.57856,112932319,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
4,739950,LT51590691994348XXX03,LT05_L1TP_159069_19941214_20170111_01_T1,LANDSAT_5,TM,1994-12-14,1,T1,1994-12-14T06:04:26.2840560Z,L1TP,159,69,28.0,-12.07291,-13.97127,47.43955,49.60335,111031951,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,14561007,LT51590691994220JSA00,LT05_L1TP_159069_19940808_20170113_01_T1,LANDSAT_5,TM,1994-08-08,1,T1,1994-08-08T06:08:14.3160560Z,L1TP,159,69,5.0,-12.06725,-13.96843,47.42019,49.58133,113221519,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
78,15146064,LT51590691996354JSA00,LT05_L1TP_159069_19961219_20170101_01_T1,LANDSAT_5,TM,1996-12-19,1,T1,1996-12-19T06:13:09.2760690Z,L1TP,159,69,32.0,-12.06190,-13.97121,47.42569,49.59238,154695791,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
79,15233663,LT51590691996210JSA00,LT05_L1TP_159069_19960728_20170103_01_T1,LANDSAT_5,TM,1996-07-28,1,T1,1996-07-28T06:05:49.9700810Z,L1TP,159,69,29.0,-12.06176,-13.96840,47.41465,49.57585,122457072,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...
80,15396645,LT51590692005266JSA00,LT05_L1TP_159069_20050923_20161124_01_T1,LANDSAT_5,TM,2005-09-23,1,T1,2005-09-23T06:38:52.4450380Z,L1TP,159,69,7.0,-12.06632,-13.95734,47.34829,49.53999,120254742,gs://gcp-public-data-landsat/LT05/01/159/069/L...,https://storage.googleapis.com/gcp-public-data...


# The Updated URL looks like

https://storage.googleapis.com/gcp-public-data-landsat/LE07/01/159/069/LE07_L1TP_159069_20191024_20191025_01_RT/LE07_L1TP_159069_20191024_20191025_01_RT_B

When downloading we will need to make a loop and add url + i + ".TIF"

### 12. Set Path & Download Data (WGET Method)

In [ ]:
import wget


for index, row in L7.iterrows():
    for x in range(1, 8):
        #print('URL is', row[19])
        url_bands = row[19] + str(x) + '.TIF'
        print(url_bands)
        
        # Define your destination folder
        satellite_path = 'YOUR_DESTINATION_FOLDER_PATH'
        # Define file name. Landsat images are saved using their product id. row[2] is PRODUCT_ID
        filename = row[2]

        destination = os.path.join(satellite_path, filename)
        os.makedirs(destination, exist_ok=True)

        # Invoke wget download method to download specified url image.
        B1_image = wget.download(url_bands, out=destination)
        url_bands
    
      

### 12.1 Alternative Request Method to Download

In [ ]:
# Import requests, shutil python module.
import requests
import shutil



for index, row in L7.iterrows():
    for x in range(1, 3):
        #print('URL is', row[19])
        url_bands = row[19] + str(x) + '.TIF'
        print(url_bands)
        
        # Define your destination folder
        satellite_path = 'YOUR_DESTINATION_FOLDER_PATH'
        # Define file name. Landsat images are saved using their product id. row[2] is PRODUCT_ID
        sub_folder = row[2] 
        filename = row[2] + '_B' + str(x) + '.TIF'

        destination = os.path.join(satellite_path, sub_folder)
        os.makedirs(destination, exist_ok=True)
        
        # Open the url image, set stream to True, this will return the stream content.
        response = requests.get(url_bands, stream=True)
        
        with open(os.path.join(destination, filename), 'wb') as output:
            shutil.copyfileobj(response.raw, output)
            del response
        
        
        
       